# Create-WFRC-MAG_GFLU

This notebook reads in the WFRC & MAG GFLU layers from AGOL, standardizes them, and merges them together into one dataset

In [1]:
import arcpy
from arcpy import env
import os
import numpy as np
from arcgis import GIS
from arcgis.features import GeoAccessor
from arcgis.features import GeoSeriesAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.pivot_table(df, values='a', index='b', columns='c', aggfunc='sum', fill_value=0)
# pd.DataFrame.spatial.from_featureclass(???)  
# df.spatial.to_featureclass(location=???,sanitize_columns=False)  

# gsa = arcgis.features.GeoSeriesAccessor(df['SHAPE'])  
# df['AREA'] = gsa.area  # KNOW YOUR UNITS

In [2]:
# fill NA values in Spatially enabled dataframes (ignores SHAPE column)
def fill_na_sedf(df_with_shape_column, fill_value=0):
    if 'SHAPE' in list(df_with_shape_column.columns):
        df = df_with_shape_column.copy()
        shape_column = df['SHAPE'].copy()
        del df['SHAPE']
        return df.fillna(fill_value).merge(shape_column,left_index=True, right_index=True, how='inner')
    else:
        raise Exception("Dataframe does not include 'SHAPE' column")

In [3]:
outputs = ['.\\Outputs', "scratch.gdb", 'Generalized_Future_Land_Use_2025.gdb']

if not os.path.exists(outputs[0]):
    os.makedirs(outputs[0])

gdb = os.path.join(outputs[0], outputs[1])
gdb2 = os.path.join(outputs[0], outputs[2])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

if not arcpy.Exists(gdb2):
    arcpy.CreateFileGDB_management(outputs[0], outputs[2])

In [4]:
wfrc_gflu = 'https://services1.arcgis.com/taguadKoI1XFwivx/arcgis/rest/services/Generalized_Future_Land_Use_%282025%29/FeatureServer/0'
mag_gflu = 'https://services2.arcgis.com/EiGeaCDLpVDPqdJ5/arcgis/rest/services/General_Plan_Land_Use_2023/FeatureServer/0'
wfrc_gflu_id = 'ae7dc08d0ea048eba4f9c1687969d426'
mag_gflu_id = '00c0c95b5d864904825073963bc5967b'

In [5]:
# Connect to ArcGIS Online or Enterprise
gis = GIS("home")

# Access a feature layer using ID
def read_from_web(id):
    item = gis.content.get(id)
    feature_layer = item.layers[0]
    sdf = pd.DataFrame.spatial.from_layer(feature_layer)
    return sdf

# Data Exploration

In [6]:
# wfrc_gflu_sdf = read_from_web(wfrc_gflu_id)
# wfrc_gflu_sdf.head(2)

In [7]:
# wfrc_gflu_sdf['GenLUType'].value_counts()

In [8]:
# mag_gflu_sdf = read_from_web(mag_gflu_id)
# mag_gflu_sdf.head(2)

In [9]:
# mag_gflu_sdf['GenLUType'].value_counts()

# Processing

In [10]:
# read in feature layers from AGOL
wfrc_gflu_lyr = arcpy.MakeFeatureLayer_management(wfrc_gflu, 'wfrc_gflu_lyr')
mag_gflu_lyr = arcpy.MakeFeatureLayer_management(mag_gflu, 'mag_gflu_lyr')

In [11]:
# dissolve feature layers
wfrc_gflu_dissolved = arcpy.management.Dissolve(wfrc_gflu_lyr, os.path.join(gdb, 'wfrc_gflu_dissolved'), ['CityLUType','City','County','GenLUType', 'MaxDUA', 'PlanSource', 'DataSource', 'AnnexArea'], "", "MULTI_PART")
mag_gflu_dissolved = arcpy.management.Dissolve(mag_gflu_lyr, os.path.join(gdb, 'mag_gflu_dissolved'), ['CityLUType','City','County','GenLUType', 'MaxDUA', 'PlanSource', 'DataSource'], "", "MULTI_PART")

# merge wfrc and mag
wfrc_mag_gflu_dissolved = arcpy.management.Merge([wfrc_gflu_dissolved, mag_gflu_dissolved], output=os.path.join(gdb, 'wfrc_mag_gflu_dissolved'))

In [15]:
# read the merged features into a dataframe
wfrc_mag_gflu_dissolved_sdf = pd.DataFrame.spatial.from_featureclass(wfrc_mag_gflu_dissolved[0])

In [16]:
# look up table  for general land use types
lu = {
'Residential SF':'Residential Single-Family',
'Residential - Low Density':'Residential Single-Family', # MAG

'Residential MF':'Residential Multi-Family',
'Residential - High Density':'Residential Multi-Family', # MAG

'Any Residential':'Any Residential',
'Residential - Medium Density':'Any Residential', # MAG

'Retail':'Retail',

'Office':'Office',

'Industrial':'Industrial',

'Any Commercial':'Mixed Commercial',
'Industrial/Office':'Mixed Commercial',
'Industrial/Retail':'Mixed Commercial',
'Retail/Office':'Mixed Commercial',
'Commercial':'Mixed Commercial',
'Business Park':'Mixed Commercial', # MAG

'Any Development':'Mixed Use',
'Industrial/Mixed Use MF':'Mixed Use',
'Mixed Use':'Mixed Use',
'Mixed Use MF':'Mixed Use',
'Mixed Use SF':'Mixed Use',
'Residential MF/Office':'Mixed Use',
'Residential SF/Retail':'Mixed Use',
'Residential/Office':'Mixed Use',
'Residential/Retail':'Mixed Use',
'Mixed Use - High Density':'Mixed Use', # MAG

'Government/Education':'Government / Education',
'Public':'Government / Education', # MAG

'NoBuild':'Parks / Protected Lands / Agriculture',
'Open Space':'Parks / Protected Lands / Agriculture', # MAG
'Residential - Agriculture':'Parks / Protected Lands / Agriculture', # MAG
}


In [17]:
# reclassify to common categories
wfrc_mag_gflu_dissolved_sdf['GenLUType2'] = wfrc_mag_gflu_dissolved_sdf['GenLUType'].map(lu)

In [18]:
# export features
wfrc_mag_gflu_dissolved_sdf.spatial.to_featureclass(location=os.path.join(gdb2, 'Generalized_Future_Land_Use_2025'),sanitize_columns=False)  

'e:\\Tasks\\Create-WFRC-MAG_GFLU\\Outputs\\Generalized_Future_Land_Use_2025.gdb\\Generalized_Future_Land_Use_2025'